+ 在线book：<https://zh-v2.d2l.ai/chapter_multilayer-perceptrons/kaggle-house-price.html>
+ jupyter对应的slides:<http://courses.d2l.ai/zh-v2/assets/notebooks/chapter_multilayer-perceptrons/kaggle-house-price.slides.html#/>